## Import Functions and Spark Config

In [1]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fe1f13bc-b63d-4a9f-aea7-14716fc04a49;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (47ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (33ms)
:: resolution report :: resolve 741ms :: artifacts dl 85ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/03/07 14:57:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/07 14:57:38 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
print('PySpark Version :'+spark.version)

PySpark Version :3.3.0


In [5]:
df = readCheckpoint(size='medium')

 3:00PM UTC on Mar 07, 2023 --- read and cache time: 0.8267385959625244 seconds ---


In [6]:
df.limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action,hostname,user_name,event_minute,event_day,event_hour
0,6deb415a-8b9a-4e71-b87f-f211a5429dea,2019-09-23 16:00:00.148,ece01c15-d27b-4995-9774-6eca17cf610c,2243a0c5-744a-4db8-8ccc-b17da6329c29,PROCESS,OPEN,SysClient0171.systemia.com,None,0,23,16
1,205b6acd-8d16-464c-a10f-8bb3090d185b,2019-09-23 16:00:00.118,1dc5145c-480d-4987-b943-35b85429dd8d,3c2b2502-2e63-4da5-9bfb-26e4ebde1427,FLOW,INFO,SysClient0115.systemia.com,None,0,23,16
2,7f1b20af-ecf1-41de-8b8a-e12b20b6aa16,2019-09-23 16:00:00.148,ece01c15-d27b-4995-9774-6eca17cf610c,2243a0c5-744a-4db8-8ccc-b17da6329c29,PROCESS,OPEN,SysClient0171.systemia.com,None,0,23,16
3,3567c018-e128-468b-82cd-6f6054d04ce0,2019-09-23 16:00:00.124,1534ff40-2ac0-4f9a-af9e-937e372eab46,3c2b2502-2e63-4da5-9bfb-26e4ebde1427,FLOW,INFO,SysClient0115.systemia.com,None,0,23,16
4,1dba1e26-d169-4f4d-aead-45059fc758a6,2019-09-23 16:00:00.151,3ca1712a-aee0-4360-8ae5-fbf7e86adfc6,ece01c15-d27b-4995-9774-6eca17cf610c,PROCESS,OPEN,SysClient0171.systemia.com,None,0,23,16


In [3]:
# Create a Vertex DataFrame with unique ID column "id"
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
], ["id", "name", "age"])
# Create an Edge DataFrame with "src" and "dst" columns
e = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
], ["src", "dst", "relationship"])
# Create a GraphFrame
from graphframes import *
g = GraphFrame(v, e)

# Query: Get in-degree of each vertex.
g.inDegrees.show()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---+--------+
| id|inDegree|
+---+--------+
|  b|       2|
|  c|       1|
+---+--------+



In [4]:
# Query: Count the number of "follow" connections in the graph.
g.edges.filter("relationship = 'follow'").count()

2

In [5]:
# Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|  c|1.8994109890559092|
|  b|1.0905890109440908|
|  a|              0.01|
+---+------------------+



In [8]:
s3_url_refined = "s3a://sapient-bucket-refined/"
s3_read_loc = f"{s3_url_refined}/prod/ecar"
df = spark.read.parquet(s3_read_loc)\
                    .filter((dayofmonth(col('timestamp')) == 23)) \
                    .cache()

In [ ]:
df.count()

## Set Environment

In [6]:
# env - dev / prod
# size - small / medium / large / all
# readCheckpoint automatically caches. use df.unpersist() to release from memory.
env='prod'
size='small'

## Data Processing

In [7]:
df_ecar = readCheckpoint('ecar', env, size).cache()

 4:33AM UTC on Mar 06, 2023 --- read and cache time: 3.26792573928833 seconds ---


In [8]:
df_ecar.limit(25).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action,hostname
0,5c3a15bc-bdca-46e3-bada-498e5a478ee3,2019-09-23 15:02:07.532,2ae55613-0147-4d2c-b52b-0edf1491e44d,7c94aca1-79c5-4880-b865-e44ebb3c28c4,FLOW,INFO,SysClient0125.systemia.com
1,71a51854-7640-4122-87a9-b97dc34f5b23,2019-09-23 15:02:07.742,844557f7-57c1-426f-8f33-e674b374b2c3,7c94aca1-79c5-4880-b865-e44ebb3c28c4,FLOW,INFO,SysClient0125.systemia.com
2,04afacdd-f053-4d62-8c49-4ad9fff8c4ab,2019-09-23 15:02:07.809,fdf101f7-fded-4eae-bfba-80b719042f18,7c94aca1-79c5-4880-b865-e44ebb3c28c4,FLOW,INFO,SysClient0125.systemia.com
3,5313313e-e19a-4f71-aaf6-3b8c57f4f61d,2019-09-23 15:02:07.853,f41f1bde-bedf-4891-82aa-e48b6aa0d87b,7c94aca1-79c5-4880-b865-e44ebb3c28c4,FLOW,INFO,SysClient0125.systemia.com
4,4846ee65-6990-4c35-b1d8-55da9587cc6f,2019-09-23 15:02:07.853,902a6d9e-85c3-47dc-98bb-4aedae291e72,7c94aca1-79c5-4880-b865-e44ebb3c28c4,FLOW,INFO,SysClient0125.systemia.com
5,dd42effc-a40a-4f77-b939-488337ca4c28,2019-09-23 15:02:07.854,6abaf4fa-ffbc-4fd6-9e18-11f4c016615f,7c94aca1-79c5-4880-b865-e44ebb3c28c4,FLOW,INFO,SysClient0125.systemia.com
6,420b7bd8-722b-42d7-9f6d-5b9c607eed45,2019-09-23 15:02:07.855,099a3e5c-afa4-4cc3-8115-573453fb6d36,7c94aca1-79c5-4880-b865-e44ebb3c28c4,FLOW,INFO,SysClient0125.systemia.com
7,1b5effb5-119a-4085-9b76-b1da5272e90a,2019-09-23 15:02:07.855,10f26cda-1067-4f67-9fbb-e6cb03870eb1,7c94aca1-79c5-4880-b865-e44ebb3c28c4,FLOW,INFO,SysClient0125.systemia.com
8,c98b7f25-c4d3-40eb-8135-bb73394c119f,2019-09-23 15:02:07.857,dfe5a399-ddce-4c22-9f1b-242a9a45b57a,7c94aca1-79c5-4880-b865-e44ebb3c28c4,FLOW,INFO,SysClient0125.systemia.com
9,dfc39da1-a20f-4163-8bec-5bd84ead0180,2019-09-23 15:02:07.866,59aa38e2-1dfb-4f09-9fcb-a66828ca65c5,7c94aca1-79c5-4880-b865-e44ebb3c28c4,FLOW,INFO,SysClient0125.systemia.com


In [5]:
df_labels = readCheckpoint('labels', env)

 8:45AM UTC on Feb 28, 2023 --- read and cache time: 0.2422952651977539 seconds ---


In [6]:
df_labels.limit(5).toPandas()

,hostname,id,objectID,actorID,timestamp,object,action
0,SysClient0955.systemia.com,3e350ce3-c4b3-4640-92d9-feaea0274cab,83f11800-88a3-4fa9-8c51-a469d66fca16,473d7954-6271-43b8-9984-702378220af2,2019-09-23T14:58:28.051-04:00,SHELL,COMMAND
1,SysClient0955.systemia.com,7a396cde-6485-40b4-baa2-8d7848fa02a3,83f11800-88a3-4fa9-8c51-a469d66fca16,473d7954-6271-43b8-9984-702378220af2,2019-09-23T14:58:33.139-04:00,SHELL,COMMAND
2,SysClient0955.systemia.com,2f33f5e3-1714-4069-850f-863680edbdef,83f11800-88a3-4fa9-8c51-a469d66fca16,473d7954-6271-43b8-9984-702378220af2,2019-09-23T14:58:33.141-04:00,SHELL,COMMAND
3,SysClient0955.systemia.com,725ad9a5-c876-4dc2-9c32-12dc6bb4da66,83f11800-88a3-4fa9-8c51-a469d66fca16,473d7954-6271-43b8-9984-702378220af2,2019-09-23T14:58:33.159-04:00,SHELL,COMMAND
4,SysClient0955.systemia.com,a8a78df2-a996-4c16-90f7-6455091eb91d,83f11800-88a3-4fa9-8c51-a469d66fca16,473d7954-6271-43b8-9984-702378220af2,2019-09-23T14:58:33.158-04:00,SHELL,COMMAND


In [7]:
df_labels.select(countDistinct("objectID")).show()

+------------------------+
|count(DISTINCT objectID)|
+------------------------+
|                   69876|
+------------------------+



In [8]:
df_labels.select(countDistinct("actorID")).show()

+-----------------------+
|count(DISTINCT actorID)|
+-----------------------+
|                    150|
+-----------------------+



In [9]:
df_labels.select("object").distinct().show()

+--------+
|  object|
+--------+
|   SHELL|
|REGISTRY|
|    FLOW|
|  THREAD|
|    FILE|
|  MODULE|
| PROCESS|
+--------+



In [10]:
df_labels.groupby("object","id").count().groupBy("object").count().orderBy(col("count").desc()).show()

+--------+------+
|  object| count|
+--------+------+
|   SHELL|165473|
|    FLOW| 80436|
|    FILE| 22485|
|  THREAD| 11283|
| PROCESS|  7908|
|  MODULE|  4570|
|REGISTRY|   109|
+--------+------+



In [11]:
df_labels.groupby("action","id").count().groupBy("action").count().orderBy(col("count").desc()).show()

+-------------+------+
|       action| count|
+-------------+------+
|      COMMAND|165473|
|      MESSAGE| 55781|
|        START| 24652|
|         READ| 10567|
|       CREATE|  8967|
|         OPEN|  7218|
|    TERMINATE|  6555|
|        WRITE|  5079|
|         LOAD|  4570|
|       RENAME|  1846|
|REMOTE_CREATE|   614|
|       MODIFY|   515|
|       DELETE|   318|
|         EDIT|    61|
|          ADD|    31|
|       REMOVE|    17|
+-------------+------+



In [12]:
df_labels.groupby("hostname","actorID","id").count().groupBy("hostname","actorID").count().orderBy(col("count").desc()).show()

+--------------------+--------------------+-----+
|            hostname|             actorID|count|
+--------------------+--------------------+-----+
|SysClient0201.sys...|a52f6519-6265-443...|25809|
|SysClient0010.sys...|3a3de01b-44d0-4a1...|19453|
|SysClient0069.sys...|a80f73dc-5713-460...|19438|
|SysClient0203.sys...|c2730252-3ab3-40f...|19433|
|SysClient0358.sys...|f247d0d1-6ce4-4f8...|19398|
|SysClient0851.sys...|266392d7-012e-479...|19352|
|SysClient0618.sys...|9211f414-ef00-4aa...|19274|
|SysClient0811.sys...|60f28094-a2f7-49e...|17706|
|SysClient0351.sys...|903916d0-e580-45e...|16857|
|SysClient0501.sys...|054df7d9-801e-49d...|13673|
|SysClient0501.sys...|281edc04-1ee7-4f6...|10561|
|SysClient0660.sys...|3eea9071-9ceb-468...| 9189|
|SysClient0402.sys...|e490633e-4e82-4bb...| 8065|
|SysClient0355.sys...|d359ffa5-8bf5-4f1...| 4621|
|SysClient0559.sys...|53efe8c9-3c95-4df...| 4396|
|SysClient0255.sys...|b5a0a459-229f-4e4...| 4393|
|SysClient0874.sys...|ee99fcf7-84fa-49f...| 4392|


In [13]:
df_ecar.select("acuity_level").distinct().show()

+------------+
|acuity_level|
+------------+
|        null|
+------------+



In [ ]:
nodes = df_ecar.select(